In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import train_test_split
import os

In [3]:
# 1. Load CIFAR-10 dataset
# CIFAR-10 is already available in TensorFlow datasets, but we can carve out validation data manually from the training data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()


170498071/170498071 [==============================] - 2s 0us/step


In [4]:
# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

In [5]:
# Split part of the training data into validation
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)


In [6]:
# 2. Build the MobileNetV2 model
base_model = MobileNetV2(input_shape=(32, 32, 3), include_top=False, weights=None)


In [7]:

base_model.trainable = True

In [8]:
# Build the custom model on top of MobileNetV2
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [9]:
# 3. Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
# 4. Create ModelCheckpoint callback to save the best model
# Save the model to Google Drive (make sure you've mounted Google Drive)
google_drive_path = '/content/drive/MyDrive/cifar10_mobilenetv2_without_imagenet_weight_best_model.keras'


In [11]:
checkpoint_callback = ModelCheckpoint(google_drive_path,
                                      save_best_only=True,
                                      monitor='val_loss',
                                      mode='min',
                                      verbose=1)

In [12]:
# 5. Train the model
history = model.fit(x_train, y_train, epochs=60, validation_data=(x_val, y_val), callbacks=[checkpoint_callback])


Epoch 1/60
1407/1407 [==============================] - ETA: 0s - loss: 1.8430 - accuracy: 0.3093
Epoch 1: val_loss improved from inf to 2.32181, saving model to /content/drive/MyDrive/cifar10_mobilenetv2_without_imagenet_weight_best_model.keras
1407/1407 [==============================] - 219s 148ms/step - loss: 1.8430 - accuracy: 0.3093 - val_loss: 2.3218 - val_accuracy: 0.1018
Epoch 2/60
1407/1407 [==============================] - ETA: 0s - loss: 1.5341 - accuracy: 0.4416
Epoch 2: val_loss did not improve from 2.32181
1407/1407 [==============================] - 205s 146ms/step - loss: 1.5341 - accuracy: 0.4416 - val_loss: 2.3439 - val_accuracy: 0.1046
Epoch 3/60
1407/1407 [==============================] - ETA: 0s - loss: 1.3832 - accuracy: 0.5055
Epoch 3: val_loss improved from 2.32181 to 2.25116, saving model to /content/drive/MyDrive/cifar10_mobilenetv2_without_imagenet_weight_best_model.keras
1407/1407 [==============================] - 203s 145ms/step - loss: 1.3832 - accurac